In [5]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential #
from tensorflow.keras.layers import Conv2D, MaxPooling2D #重頭戲，3*3矩陣為2D所以是conv2D，如果是向量的話就是con1D
from tensorflow.keras.layers import Dense, Flatten #flatten拉平？
from tensorflow.keras.optimizers import SGD 

### 1. 讀入 MNSIT 數據集

In [10]:
from tensorflow.keras.datasets import mnist 

In [11]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() √

SyntaxError: ignored

### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [12]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255 #三原色-->三個channel（三個矩陣）-->(60000,28,28,3)

In [13]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [14]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [18]:
model = Sequential()

In [19]:
model.add(Conv2D(8, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [20]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [21]:
model.add(Conv2D(16, (3,3), padding='same',
                activation='relu'))

In [22]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [24]:

model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [26]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [29]:
model.add(Flatten()) #把自分板拉平

In [30]:
model.add(Dense(20, activation='relu'))

In [32]:
model.add(Dense(10, activation='softmax')) #輸出一定要十個

#### 看一下我們的神經網路

In [33]:
model.summary() #8個3*3的filter，總共有(3*3+1)*8=80個權重

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 8)         80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          9280      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 32)          18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 32)          0

#### 組裝

In [34]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [36]:
model.fit(x_train, y_train, batch_size=128, epochs=12)

Epoch 1/12
469/469 [==============================] - 41s 87ms/step - loss: 0.0900 - accuracy: 0.1286
Epoch 2/12
469/469 [==============================] - 40s 85ms/step - loss: 0.0900 - accuracy: 0.1130
Epoch 3/12
469/469 [==============================] - 40s 85ms/step - loss: 0.0900 - accuracy: 0.1124
Epoch 4/12
469/469 [==============================] - 40s 85ms/step - loss: 0.0900 - accuracy: 0.1124
Epoch 5/12
469/469 [==============================] - 40s 84ms/step - loss: 0.0900 - accuracy: 0.1124
Epoch 6/12
469/469 [==============================] - 39s 84ms/step - loss: 0.0900 - accuracy: 0.1124
Epoch 7/12
469/469 [==============================] - 40s 85ms/step - loss: 0.0900 - accuracy: 0.1124
Epoch 8/12
469/469 [==============================] - 41s 88ms/step - loss: 0.0900 - accuracy: 0.1124
Epoch 9/12
469/469 [==============================] - 39s 83ms/step - loss: 0.0899 - accuracy: 0.1124
Epoch 10/12
469/469 [==============================] - 39s 83ms/step - loss: 0.089

### Step 3. 預測

In [37]:
result = model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


### 看看測試資料表現如何

In [38]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 3s 11ms/step - loss: 0.0899 - accuracy: 0.1135


In [39]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 11.35%


In [40]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [41]:
from ipywidgets import interact_manual

In [42]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [43]:
model.save('myCNNmodel.h5')